_Run the first 2 cells without modifications_

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython import get_ipython
from IPython.display import Markdown #, IFrame
# for presentations:
#display(HTML("<style>.container { width:100% !important; }</style>"))

import sys
from pathlib import Path
print('Python ver: {}\nPython env: {}'.format(sys.version, Path(sys.prefix).name))
print('Currrent dir: {}\n'.format(Path.cwd()))


def add_to_sys_path(this_path, up=False):

    """
    Prepend this_path to sys.path.
    If up=True, path refers to parent folder (1 level up).
    """

    if up:
        newp = str(Path(this_path).parent)
    else:
        newp = str(Path(this_path))

    if newp not in sys.path:
        sys.path.insert(1, newp)
        print('Path added to sys.path: {}'.format(newp))


import numpy as np
import pandas as pd
#pd.set_option("display.max_colwidth", 200)
import matplotlib as mpl
from matplotlib import pyplot as plt
#plt.ion()
plt.style.use('seaborn-v0_8-muted')
from pprint import pprint as pp

def fdir(obj, start_with_str='_', exclude=True):
    """Filtered dir() for method discovery."""
    return [d for d in dir(obj) if not d.startswith(start_with_str) == exclude]


def new_section(title='New section'):
    style = "text-align:center;background:#c2d3ef;padding:16px;color:#ffffff;font-size:2em;width:98%"
    return HTML('<div style="{}">{}</div>'.format(style, title))


def add_div(div_class, div_start, div_text, output_string=True):
    """
    Behaviour with default `output_string=True`:
    The cell is overwritten with the output string, but the cell mode is still in 'code' not 'markdown':
    ```
    [x]
    add_div('alert-warning', 'Tip: ', 'some tip here', output_string=True)
    [x]
    <div class="alert alert-warning"><b>Tip: </b>some tip here</div>
    ```
    The only thing to do is change the cell mode to Markdown.
    If `output_string=False`, the HTML output is displayed in an output cell.
    """

    accepted = ['alert-info', 'alert-warning', 'alert-danger']

    if div_class not in accepted:
        return HTML(f"""<div class="alert"><b>Wrong class:</b> `div_start` is one of {accepted}.
                    </div>""")
    div = f"""<div class="alert {div_class}"><b>{div_start}</b>{div_text}</div>"""
    if output_string:
        return get_ipython().set_next_input(div, 'markdown')
    else:
        return Markdown(div) #HTML(div)



# autoreload extension
ipython = get_ipython()
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

Python ver: 3.11.5 | packaged by conda-forge | (main, Aug 27 2023, 03:34:09) [GCC 12.3.0]
Python env: mce
Currrent dir: /home/cat/projects/MCCE_Benchmarking/notebooks



In [5]:
add_to_sys_path(Path.cwd(), up=True)

In [4]:
from benchmark import audit
import shutil
import random

In [13]:
# parser.add_argument("prot", metavar="pdb", nargs=1)  and:

common_options = {
          "msg":"MCCE benchmarking - Common options.",
          "--norun": {"default":False, "help":"Create run.prm but do not run step 1.", "action":"store_true"},
          "-e":      {"metavar":"/path/to/mcce", "default":"mcce", "help":"mcce executable location; default: %(default)s."},
          # -d :: epsilon in S1-S3; -d is titration interval in S4.
          "-d3":      {"metavar":"epsilon", "default":"4.0", "help":"protein dielectric constant; default: %(default)s."},
          "-u":      {"metavar":"Key=Value list", "default":"",
                     "help":"""Any comma-separated KEY=var from run.prm; e.g.:
                     -u HOME_MCCE=/path/to/mcce_home,H2O_SASCUTOFF=0.05,EXTRA=./extra.tpl; default: %(default)s."""},
}


CLI_NAME = "mcce_bench"  # as per pyproject.toml

USAGE = f"{CLI_NAME} <sub-command for step to run> <args for step>\n"

DESC = """
    Run the 'MCCE_CDC pipeline' in 3 steps:
      step 1. ms sampling to pdbs
      step 2. mcce to gromacs pdbs conversion
      step 3. output files for sites energies (.npy) and ms matrices (.csv)

    The command for the pipeline is `mccecdc`, which expects a sub-command,
    one among `step1`, `step2` or `step3`, then the argument(s) for each.

    Output folders:
      pdbs_dir = mcce_dir/SAMPLED_PDBS
      parsed_dir = mcce_dir/PARSED_PDBS
      - These module variables hold the folder names:
          SAMPLED_PDBS = "pdb_output_mc"
          PARSED_PDBS = "parsed_pdb_output_mc"

"""

def bench_parser():
    """Command line arguments parser with sub-commands for use in benchmarking.
    """

    def arg_valid_dirpath(p: str):
        """Return resolved path from the command line."""
        if not len(p):
            return None
        return Path(p).resolve()

    p = ArgumentParser(
        prog = f"{CLI_NAME} ",
        description = DESC,
        usage = USAGE,
        formatter_class = RawDescriptionHelpFormatter,
        epilog = ">>> END of %(prog)s.",
    )
    subparsers = p.add_subparsers(required=True,
                                  title='pipeline step commands',
                                  description='Subcommands of the MCCE-CDC processing pipeline',
                                  help='The 3 steps of the MCCE-CDC processing pipeline',
                                  dest='subparser_name'
                                 )

    # do_ms_to_pdbs
    step1 = subparsers.add_parser('step1',
                                  formatter_class = RawDescriptionHelpFormatter,
                                  help=HELP_1)
    step1.add_argument(
        "mcce_dir",
        type = arg_valid_dirpath,
        help = "The folder with files from a MCCE simulation; required.",
    )
    step1.add_argument(
        "sample_size",
        type = int,
        help = "The size of the microstates sample, hence the number of pdb files to write; required",
    )
    step1.add_argument(
        "-msout_file",
        type = str,
        default = "pH7eH0ms.txt",
        help = "Name of the mcce_dir/ms_out/ microstates file, `pHXeHYms.txt'; default: %(default)s.""",
    )
    step1.add_argument(
        "-sampling_kind",
        type = str,
        choices = ["d", "deterministic", "r", "random"],
        default = "r",
        help = """The sampling kind: 'deterministic': regularly spaced samples,
        'random': random indices over the microstates space; default: %(default)s.""",
    )
    step1.add_argument(
        "-seed",
        type = int,
        default = None,
        help = "The seed for random number generation. Only applies to random sampling; default: %(default)s.",
    )
    step1.set_defaults(func=do_ms_to_pdbs)

    # do_convert_pdbs
    step2 = subparsers.add_parser('step2',
                                  formatter_class = RawDescriptionHelpFormatter,
                                  help=HELP_2)
    step2.add_argument(
        "mcce_dir",
        type = arg_valid_dirpath,
        help = "The folder with files from a MCCE simulation; required.",
    )
    step2.add_argument(
        "-empty_parsed_dir",
        type = bool,
        default = True,
        # folder reuse:
        help = "If True, the pdb files in the folder `parsed_dir` will be deleted before the new conversion."
    )
    step2.set_defaults(func=do_convert_pdbs)

    # do_site_energies + matrices
    step3 = subparsers.add_parser('step3',
                                  formatter_class = RawDescriptionHelpFormatter,
                                  help=HELP_3)
    step3.add_argument(
        "mcce_dir",
        type = arg_valid_dirpath,
        help = "The folder with files from a MCCE simulation; required.",
    )
    # Remove? Current cofactor of interest is "CLA" (hard-coded in `microstates_sites_energies`).
    step3.add_argument(
        "-cofactor_list",
        type = list,
        default = cofactors_list,
        help="List of cofactors (3-char string) found in the pdb used in the MCC simulation; default: %(default)s.",
    )
    step3.set_defaults(func=do_site_energies)

    return p



mcce_step_options = {
    "S1":{"msg":"Run mcce step 1, premcce to format PDB file to MCCE PDB format.",
          "--noter": {"default":False, "help":"Do not label terminal residues (for making ftpl).", "action":"store_true"},
          "--dry":   {"default":False, "help":"Delete all water molecules.", "action":"store_true"},
          },
    "S2":{"msg":"Run mcce step 2, make side chain conformers from step1_out.pdb.",
          "-l":      {"metavar":"level", "default":1, "help":"Conformer level 1=quick (default), 2=medium, 3=full", "type":int},
          },
    "S3":{"msg":"Run mcce step 3, energy calculations, with multiple threads.",
          # should have been --r:
          "-r":      {"default":False, "help":"refresh opp files and head3.lst without running delphi", "action":"store_true"},
          "-c":      {"metavar":"('conf start', 'conf end')", "default":[1, 99999], "nargs":2,
                       "help":"starting and ending conformer, default to 1 and 9999", "type":int},
          "-f":      {"metavar":"tmp folder", "default":"/tmp", "hel":"delphi temporary folder, default to /tmp"},
          "-p":      {"metavar":"processes", "default":1, "help":"run mcce with number of processes; default: %(default)s.", "type":int},
          },
    "S4":{"msg":"Run mcce step 4, Monte Carlo sampling to simulate a titration.",
          "--xts":   {"default":False, "help":"Enable entropy correction, default is false", "action":"store_true"},
          "--ms":    {"default":False, "help":"Enable microstate output", "action":"store_true"},
          "-t":      {"metavar":"ph or eh", "default":"ph", "help":"titration type: pH or Eh."},
          "-i":      {"metavar":"initial ph/eh", "default":"0.0", "help":"Initial pH/Eh of titration; default: %(default)s."},
          "-d":      {"metavar":"interval", "default":"1.0", "help":"titration interval in pJ or mV; default: %(default)s."},
          "-n":      {"metavar":"steps", "default":"15", "help":"number of steps of titration; default: %(default)s."},
          }
}


In [14]:
pp(mcce_step_options)

{'S1': {'--dry': {'action': 'store_true',
                  'default': False,
                  'help': 'Delete all water molecules.'},
        '--norun': {'action': 'store_true',
                    'default': False,
                    'help': 'Create run.prm but do not run step 1.'},
        '--noter': {'action': 'store_true',
                    'default': False,
                    'help': 'Do not label terminal residues (for making '
                            'ftpl).'},
        '-d': {'default': '4.0',
               'help': 'protein dielectric constant; default: %(default)s.',
               'metavar': 'epsilon'},
        '-e': {'default': 'mcce',
               'help': 'mcce executable location; default: %(default)s.',
               'metavar': '/path/to/mcce'},
        '-u': {'default': '',
               'help': 'Any comma-separated KEY=var from run.prm; e.g.:\n'
                       '                     -u '
                       'HOME_MCCE=/path/to/mcce_home,H2O_SASCU

---
---

# Prep of the "master" pdbs folder, `PDBS`:
 * Remove any MCCE output files or folder along with prot.pdb
---

In [6]:
DATA = Path.cwd().parent.joinpath("data")

PDBS = DATA.joinpath("clean_pdbs")
PDBS
WT = DATA.joinpath("WT_pkas.csv")
WT
Q_BOOK = DATA.joinpath("book.txt")
Q_BOOK
PROTS = DATA.joinpath("proteins.txt")
PROTS

PosixPath('/home/cat/projects/MCCE_Benchmarking/data/clean_pdbs')

PosixPath('/home/cat/projects/MCCE_Benchmarking/data/WT_pkas.csv')

PosixPath('/home/cat/projects/MCCE_Benchmarking/data/book.txt')

PosixPath('/home/cat/projects/MCCE_Benchmarking/data/proteins.txt')

In [35]:
def main():
    """main fn of pkanalysis.py"""

    calc_pkas = job_pkas_to_dict()
    expr_pkas = experimental_pkas_to_dict()
    matched_pks = match_pkas(expr_pkas, calc_pkas)
    n = len(matched_pks)
    matched_pkas_to_csv(matched_pks)

    # Overall fitting
    x = np.array([p[1] for p in matched_pks])  # x: experiemntal pKas
    y = np.array([p[2] for p in matched_pks])  # y: calculated pKas

    b, m = np.polynomial.Polynomial.fit(x, y, 1, domain=[0,20]).convert().coef
    op = "+" if m > 0 else "-"
    print(f"y (calculated pKa) = {b:.3f} {op} {abs(m):.3f}x (experimental pKa)")

    delta = x - y
    rmsd = np.sqrt(np.mean(delta**2))
    print(f"RMSD between expl. and calc. = {rmsd:.3f}")

    within_2, within_1 = 0, 0
    for d in np.abs(delta):
        if d <= 2.0:
            within_2 += 1
            if d <= 1.0:
                within_1 += 1

    print(f"{within_2/n:.1%} within 2 pH units")
    print(f"{within_1/n:.1%} within 1 pH unit")


## tests

In [95]:
matched_pks = []
for i in range(10):
    matched_pks.append((random.choice("ABCDRGWSX"),
                         random.choice([3.2, 5.1, 6., 4.4, 7.2]),
                        random.choice([3.2, 5.1, 6., 4.4, 7.2]*2)))
matched_pks

[('R', 3.2, 3.2),
 ('W', 5.1, 7.2),
 ('B', 3.2, 4.4),
 ('C', 4.4, 7.2),
 ('A', 4.4, 7.2),
 ('D', 6.0, 7.2),
 ('W', 7.2, 7.2),
 ('R', 4.4, 6.0),
 ('W', 3.2, 5.1),
 ('W', 3.2, 3.2)]

In [33]:
pka_dict = experimental_pkas_to_dict(WT)
len(pka_dict)
list(pka_dict.keys())[:10]

1214

[('135L', 'ASP-A0018_'),
 ('135L', 'GLU-A0035_'),
 ('135L', 'GLU-A0007_'),
 ('135L', 'ASP-A0119_'),
 ('135L', 'ASP-A0087_'),
 ('135L', 'ASP-A0052_'),
 ('1A2P', 'CTR-C0110_'),
 ('1A2P', 'HIS+C0102_'),
 ('1A2P', 'ASP-C0101_'),
 ('1A2P', 'ASP-C0086_')]